In [13]:
import pandas_ta as ta
import numpy as np
import pandas as pd
import pandas_ta as ta
import tushare as ts
import jqdatasdk
import os
import numpy as np
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [14]:
# Obtain Data
df_300 = pd.read_csv("000300_daily.csv", header=0, names=["DateTime", "Open", "Close", "High", "Low", "Volume", "Money"])
df_300.head()

,DateTime,Open,Close,High,Low,Volume,Money
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09


In [15]:
# 继续对CSI300_DataFrame进行预处理 计算出每日收盘价Close差价, 以及每日变化百分比
# Pre_Close --> 对应昨日收盘价
# Change    --> 两日收盘价之差 今日 - 昨日
# Pct_Change--> 每日收盘价相较于前一天变化百分比
df_300['Pre_Close'] = df_300['Close'].shift(1)
df_300['Pre_Close'].fillna(950, inplace=True)
df_300['Change'] = df_300['Close'] - df_300['Pre_Close']
df_300['Pct_Change'] = round((df_300['Change'] / df_300['Pre_Close']) * 100,2)

In [16]:
# 检查预处理结束的df_300 dataframe 
# data feed ready 进入策略部分
# 仍需要继续处理Aroon部分数据添加column title
# Aroon_down // Aroon_up // Aroon // Signal // Position // Hold_ret // Cum_hold_ret
df_300

,DateTime,Open,Close,High,Low,Volume,Money,Pre_Close,Change,Pct_Change
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,950.00,-5.44,-0.57
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,944.56,3.31,0.35
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,947.87,4.85,0.51
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,952.72,-16.11,-1.69
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,936.61,16.15,1.72
...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2635.79,28.51,1.08
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2664.30,-9.90,-0.37
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2654.40,26.67,1.00
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2681.07,-75.96,-2.83


In [17]:
# 通过pd.DataFrame().ta.indicators()查看 Technical Indicators for Aroon
pd.DataFrame().ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [19]:
# set n = 20
n = 20
high = df_300['High']
low = df_300['Low']
# Calculate indicators for Aroon
# take defined arguments as high, low & length
aroon = ta.aroon(high, low, length = n)

# 将indicators添加到 df_300 DataFrame 中
df_300['Aroon_up'] = aroon['AROONU_{}'.format(n)] # U --> Up
df_300['Aroon_down'] = aroon['AROOND_{}'.format(n)] # D --> Down
df_300['AROON'] = aroon['AROONOSC_{}'.format(n)]
df_300  # 指标indicators也添加进了DataFrame中 参数 n = 20


,DateTime,Open,Close,High,Low,Volume,Money,Pre_Close,Change,Pct_Change,Aroon_up,Aroon_down,AROON
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,950.00,-5.44,-0.57,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,944.56,3.31,0.35,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,947.87,4.85,0.51,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,952.72,-16.11,-1.69,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,936.61,16.15,1.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2635.79,28.51,1.08,80.0,5.0,75.0
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2664.30,-9.90,-0.37,75.0,0.0,75.0
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2654.40,26.67,1.00,70.0,0.0,70.0
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2681.07,-75.96,-2.83,100.0,0.0,100.0


In [24]:
# df_300 dataframe处理完毕 下一步计算所有信号
# 计算信号
def signal_Generation(df):
    # 定义variables
    aroonup = df['Aroon_up']
    aroondown = df['Aroon_down']
    aroon = df['AROON']

    # 计算信号signals
    signals = [] # 后期append添加 初始化空列表

    # 6个list for循环遍历 
    for up, down, aron, shift_up, shift_down, shift_aroon in zip(aroonup, aroondown, aroon, aroonup.shift(1), aroondown.shift(1), aroon.shift(1)):
        signal = None # line中每条数据初始无信号显示

        if shift_up < 70 and up >= 70 and aron > 0:
            signal = 1 # open position
        
        elif shift_down < 70 and down >= 70 and aron < 0:
            signal = -1 # short position

        elif shift_up >= 50 and up < 50 and aron < 0:
            signal = -1 # short position

        elif shift_down >= 50 and down < 50 and aron > 0:
            signal = 1 # open position
        
        # 每遍历完一次 添加信号到list当中
        signals.append(signal)
    
    # 结束循环 赋值信号列表
    df['Signal'] = signals
    return df


In [25]:
df_300 = signal_Generation(df_300)


In [27]:
# check df_300 after signal processing
df_300
# we can see signals have been added to the data frame 
df_300.to_csv("check_signal.csv")

In [29]:
df_300

,DateTime,Open,Close,High,Low,Volume,Money,Pre_Close,Change,Pct_Change,Aroon_up,Aroon_down,AROON,Signal
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,950.00,-5.44,-0.57,NaN,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,944.56,3.31,0.35,NaN,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,947.87,4.85,0.51,NaN,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,952.72,-16.11,-1.69,NaN,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,936.61,16.15,1.72,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2635.79,28.51,1.08,80.0,5.0,75.0,NaN
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2664.30,-9.90,-0.37,75.0,0.0,75.0,NaN
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2654.40,26.67,1.00,70.0,0.0,70.0,NaN
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2681.07,-75.96,-2.83,100.0,0.0,100.0,NaN


In [30]:
# After defining the strategy, now we need to calculate position
# 有close之后 第二天才操作open or short position
# 所以需要shift(1)
df_300['Position'] = df_300['Signal'].fillna(method = 'ffill').shift(1).fillna(0)
# Position就不会出现NaN
df_300

,DateTime,Open,Close,High,Low,Volume,Money,Pre_Close,Change,Pct_Change,Aroon_up,Aroon_down,AROON,Signal,Position
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,950.00,-5.44,-0.57,NaN,NaN,NaN,NaN,0.0
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,944.56,3.31,0.35,NaN,NaN,NaN,NaN,0.0
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,947.87,4.85,0.51,NaN,NaN,NaN,NaN,0.0
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,952.72,-16.11,-1.69,NaN,NaN,NaN,NaN,0.0
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,936.61,16.15,1.72,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2635.79,28.51,1.08,80.0,5.0,75.0,NaN,1.0
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2664.30,-9.90,-0.37,75.0,0.0,75.0,NaN,1.0
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2654.40,26.67,1.00,70.0,0.0,70.0,NaN,1.0
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2681.07,-75.96,-2.83,100.0,0.0,100.0,NaN,1.0


In [31]:
# 统计回测表现
def statistic_performance(df_300, r0 = 0.023, data_period=1588): # Data period is 1588 rows
    position = df_300['Position']
    
    #     holding_Return :            持仓收益
    #     W_L_forHolding :            持仓胜负
    #     cum_HoldingReturn :         累计持仓收益
    #     drawback :                  回撤
    #     excess_HoldingReturn :      超额收益
     
    holding_Return = df_300['Pct_Change']/100 * position         # 持仓收益
    W_L_forHolding = holding_Return > 0                          # 返回boolean 收益率大于0即为Win
    cum_HoldingReturn = (1+holding_Return).cumprod() - 1         # 累积持仓收益
    drawback = (cum_HoldingReturn.cummax()-cum_HoldingReturn)/(1+cum_HoldingReturn).cummax()    
    excess_HoldingReturn= holding_Return-r0 / (2520*1440/data_period) # Data period is 1588 rows

    df_300['holding_Return'] = holding_Return              # 持仓收益
    df_300['W_L_forHolding'] = W_L_forHolding              # 持有期间胜负
    df_300['cum_HoldingReturn'] = cum_HoldingReturn        # 持有期间累计收益率
    df_300['drawback'] = drawback                          # 回测
    df_300['excess_HoldingReturn'] = excess_HoldingReturn  # 持仓超额收益
    

    '''多仓指标'''
    #     num_LongPos_open：                多仓开仓次数
    #     num_LongPos_win：                 多仓开仓盈利次数
    #     num_LongHolding_period：          多仓持有周期数
    #     num_LongHolding_Windperiod：      多仓持有盈利周期数
    
    '''空仓指标'''
    #     num_ShortPos_open：               空仓开仓次数
    #     num_ShortPos_win：                空仓开仓盈利次数
    #     num_ShortHolding_period：         空仓持有周期数
    #     num_ShortHolding_Winperiod：      空仓持有盈利周期数
    
    '''持仓指标'''
    #     num_HoldingPeriod：               持仓周期数（最后一笔未平仓订单也算）
    #     num_HoldingPeriod_win：           持仓盈利周期数（最后一笔未平仓订单也算）

    '''回撤 & 年化收益 & 标准差 & Shape Ratio'''
    #     Cum_Return：                      累计持仓收益
    #     max_Drawback：                    最大回撤
    #     annual_STD：                      年化标准差
    #     annual_Return：                   年化收益
    #     sharpRatio：                      夏普

    Cum_Return = cum_HoldingReturn.tolist()[-1]

    num_LongPos_open= 0 
    num_LongPos_win = 0
    num_LongHolding_period = 0
    num_LongHolding_Windperiod = 0
    num_ShortPos_open= 0 
    num_ShortPos_win = 0
    num_ShortHolding_period = 0
    num_ShortHolding_Winperiod = 0
    for w, r, pre_pos, pos in zip(W_L_forHolding, holding_Return, position.shift(1), position):
        # 有换仓（先结算上一次持仓，再初始化本次持仓）
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是第一次循环，无需结算，初始化holding position
            if pre_pos == pre_pos:
                # 结算上一次holding
                if pre_pos>0:
                    num_LongPos_open += 1
                    num_LongHolding_period += tmp_hold_period
                    num_LongHolding_Windperiod += tmp_holdWinPeriod
                    if tmp_holdingReturn>0:
                        num_LongPos_win+=1
                elif pre_pos<0:
                    num_ShortPos_open += 1      
                    num_ShortHolding_period += tmp_hold_period
                    num_ShortHolding_Winperiod += tmp_holdWinPeriod
                    if tmp_holdingReturn>0:                    
                        num_ShortPos_win+=1
            # initialize first holding
            tmp_holdingReturn = r
            tmp_hold_period = 0
            tmp_holdWinPeriod = 0
        else: # 没有换仓
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_holdWinPeriod += 1
                if abs(r)>0:
                    tmp_holdingReturn = (1+tmp_holdingReturn)*(1+r)-1       

    num_HoldingPeriod = (abs(position)>0).sum()
    num_HoldingPeriod_win = (holding_Return>0).sum()
    max_Drawback = drawback.max()    
    annual_Return = pow( 1+Cum_Return, 
                      1/(data_period/1440*len(df_300)/250) )-1
    annual_STD = excess_HoldingReturn.std() * np.sqrt(250*1440/data_period) 
    sharpRatio= annual_Return / annual_STD

    """生成回测结果 DataFrame"""
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓胜率', '空仓平均持有期', 
                        '日胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益', '年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(Cum_Return),
                          num_LongPos_open, '{:.2%}'.format(num_LongPos_win/num_LongPos_open), 
                                            '{:.2f}'.format(num_LongHolding_period/num_LongPos_open),
                          num_ShortPos_open, '{:.2%}'.format(num_ShortPos_win/num_ShortPos_open), 
                                            '{:.2f}'.format(num_ShortHolding_period/num_ShortPos_open),
                          '{:.2%}'.format(num_HoldingPeriod_win/num_HoldingPeriod), 
                          '{:.2%}'.format(max_Drawback), 
                          '{:.2f}'.format(annual_Return/max_Drawback),
                          '{:.2%}'.format(annual_Return), 
                          '{:.2%}'.format(annual_STD), 
                          '{:.2f}'.format(sharpRatio)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    return df_300, performance_df

In [32]:
statistic_performance(df_300, r0 = 0.023, data_period=1588)

(        DateTime     Open    Close     High      Low        Volume  \
 0     2005-09-01   928.52   944.56   944.95   926.06  2.230638e+09   
 1     2005-09-02   945.98   947.87   947.92   941.31  1.861444e+09   
 2     2005-09-05   949.08   952.72   952.99   944.63  1.784332e+09   
 3     2005-09-06   953.41   936.61   956.30   934.97  2.122226e+09   
 4     2005-09-07   934.99   952.76   952.90   932.65  1.960847e+09   
 ...          ...      ...      ...      ...      ...           ...   
 1583  2012-03-09  2644.10  2664.30  2664.41  2631.41  5.525892e+09   
 1584  2012-03-12  2663.24  2654.40  2666.43  2636.42  6.232722e+09   
 1585  2012-03-13  2653.68  2681.07  2681.33  2649.17  5.840447e+09   
 1586  2012-03-14  2694.47  2605.11  2705.75  2595.34  1.084265e+10   
 1587  2012-03-15  2602.66  2585.55  2618.25  2575.44  7.256596e+09   
 
              Money  Pre_Close  Change  Pct_Change  Aroon_up  Aroon_down  \
 0     1.055868e+10     950.00   -5.44       -0.57       NaN         N

In [33]:
# 回测结果可视化
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [ ]:
def visualize_performance(mkt_data):
    mkt_data['trade_datetime'] = mkt_data['DateTime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    dt = mkt_data['trade_datetime']

    f1 = figure(height=300, width=700, 
                sizing_mode='stretch_width', 
                title='Target Trend',
                x_axis_type='datetime',
                x_axis_label="trade_datetime", y_axis_label="close")
    f2 = figure(height=200, sizing_mode='stretch_width', 
                title='Position',
                x_axis_label="trade_datetime", y_axis_label="position",
                x_axis_type='datetime',
                x_range=f1.x_range)
    f3 = figure(height=200, sizing_mode='stretch_width', 
                title='Return',
                x_axis_type='datetime',
                x_range=f1.x_range)
    f4 = figure(height=200, sizing_mode='stretch_width', 
                title='Drawback',
                x_axis_type='datetime',
                x_range=f1.x_range)

    # 绘制行情
    close = mkt_data['Close']
    cumu_hold_close = (mkt_data['cum_HoldingReturn']+1)
    f1.line(dt, close/close.tolist()[0], line_width=1)
    f1.line(dt, cumu_hold_close, line_width=1, color='red')
    
    # 绘制仓位
    position = mkt_data['Position']
    f2.step(dt, position)

    # 绘制收益
    hold_r = mkt_data['holding_Return']
    f3.vbar(x=dt, top=hold_r)

    # 绘制回撤
    drawdown = mkt_data['drawback']
    f4.line(dt, -drawdown, line_width=1)

    #p = column(f1,f2,f3,f4)
    p = gridplot([ [f1],
                   #[indi],
                   [f2], 
                   [f3],
                   [f4]
                 ])
    show(p)